In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import pandas as pd
import pandas_datareader.data as web
import sklearn.mixture as mix
from pytrends.request import TrendReq
import time
import numpy as np
import scipy.stats as scs

import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
%matplotlib inline


# conda env create -f environment.yml
import urllib.request, json 
from requests.exceptions import ConnectionError
#mport plotly.plotly as py
#import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#cf.go_offline
init_notebook_mode(connected=True)
import datetime
import plotly
#print(plotly.__version__)
import plotly.io as pio
pio.templates.default = "none"

import seaborn as sns
import missingno as msno
from tqdm import tqdm
p=print

In [3]:
def GoogleTrend(keywordsList = ["Recession"],geo=''):
    pytrend = TrendReq(hl='en-US', tz=360)
    dataset = []
    pytrend.build_payload(
        kw_list=keywordsList,
        cat=0,
        timeframe='2004-01-01 '+ pd.datetime.today().strftime("%Y-%m-%d"),
        geo=geo) #geo='US'
    data = pytrend.interest_over_time()
    if not data.empty:
        data = data.drop(labels=['isPartial'],axis='columns')
        dataset.append(data)
    google_trend = pd.concat(dataset, axis=1)
    google_trend = google_trend.resample('D').fillna('ffill')
    #google_trend.to_csv('Recession_GoogleTrends_'+pd.datetime.today().strftime("%Y-%m-%d")+'.csv')
    return google_trend

In [4]:
google_trend = GoogleTrend(keywordsList = ["Recession"],geo='')
google_trend_US = GoogleTrend(keywordsList = ["Recession"],geo='US')

In [5]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=google_trend.index,
    y=google_trend["Recession"], 
    name="Recession - World",
    mode="markers+lines"))
fig.add_trace(go.Scatter(
    x=google_trend_US.index,
    y=google_trend_US["Recession"], 
    name="Recession - US",
    mode="markers+lines"))
fig.update_layout(title={'text': "Number of Google searches for 'Recession' globally and the US"})
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [6]:
# get fed data

f1 = 'TEDRATE' # ted spread
f2 = 'T10Y2Y' # constant maturity ten yer - 2 year
f3 = 'T10Y3M' # constant maturity 10yr - 3m
f4 = 'UMCSENT' # consumer sentiment
f5 = 'RECPROUSM156N' # Smoothed U.S. Recession Probabilities
f6 = 'UNEMPLOY' # Unemployment level 
f7 = 'VIXCLS' # VIX
f8 = 'A053RC1Q027SBEA' # Corporate profits before tax (without IVA and CCAdj
f9 = 'IPMAN' # Industrial Manufacturing
start = pd.to_datetime('2002-01-01')
end = pd.datetime.today()

mkt = 'SPY'
MKT = (web.DataReader([mkt], 'yahoo', start, end)['Adj Close']
       .rename(columns={mkt:mkt})
       .assign(sret=lambda x: np.log(x[mkt]/x[mkt].shift(1)))
       .dropna())
#p(MKT)
data = (web.DataReader([f2,f3], 'fred', start, end)
        .join(MKT, how='inner')
        .dropna()
       )
rec_prob = web.DataReader([f5], 'fred', start, end).dropna()
rec_prob= rec_prob.resample('D').fillna('ffill')
rec_prob.rename(columns={'RECPROUSM156N':'recession prob'}, inplace=True)

unemployment = web.DataReader([f6], 'fred', start, end).dropna()
unemployment = unemployment.resample('D').fillna('ffill')
unemployment.rename(columns={'UNEMPLOY':'Unemployment'}, inplace=True)

VIX = web.DataReader([f7], 'fred', start, end).dropna()
VIX = VIX.resample('D').fillna('ffill')
VIX.rename(columns={'VIXCLS':'VIX'}, inplace=True)

Corp_PBT = web.DataReader([f8], 'fred', start, end).dropna()
Corp_PBT = Corp_PBT.resample('D').fillna('ffill')
Corp_PBT = Corp_PBT.pct_change()
Corp_PBT.rename(columns={'A053RC1Q027SBEA':'Corp_PBT'}, inplace=True)

Ind_Manufac = web.DataReader([f9], 'fred', start, end).dropna()
Ind_Manufac = Ind_Manufac.resample('D').fillna('ffill')
Ind_Manufac = Ind_Manufac.pct_change()
Ind_Manufac.rename(columns={'IPMAN':'Ind_Manufac'}, inplace=True)



data = (data
        .join(Ind_Manufac, how='inner')
        .dropna()
       )
'''
data = (data
        .join(Corp_PBT, how='inner')
        .dropna()
       )
'''
data = (data
        .join(VIX, how='inner')
        .dropna()
       )
data = (data
        .join(unemployment, how='inner')
        .dropna()
       )
data = (data
        .join(rec_prob, how='inner')
        .dropna()
       )
data = (data
        .join(google_trend, how='inner')
        .dropna()
       )

In [7]:
# gives us a quick visual inspection of the data
#msno.matrix(data)

In [8]:
# code adapted from http://hmmlearn.readthedocs.io
# for sklearn 18.1

col = 'sret'
select = data.loc[:].dropna()

ft_cols = [f2,'Recession','recession prob','Unemployment','VIX','sret'] #'Corp_PBT'
X = select[ft_cols].values

NumRegimes = 4
model = mix.GaussianMixture(n_components=NumRegimes, 
                            covariance_type="full", 
                            n_init=170, 
                            random_state=42).fit(X)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(X)

print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covariances_[i]))
    print()

Means and vars of each hidden state
0th hidden state
mean =  [ 1.64791173e+00  4.95135614e+01  5.42133043e+01  1.06556510e+04
  3.13037539e+01 -1.29452311e-03]
var =  [5.43437018e-01 7.51023874e+02 1.95487422e+03 9.96515632e+06
 1.85861621e+02 5.27683837e-04]

1th hidden state
mean =  [1.92196638e+00 1.65970217e+01 1.15931832e-01 1.19395145e+04
 1.88864105e+01 7.04697725e-04]
var =  [3.94474858e-01 4.77909023e+01 1.10412570e-02 5.38873997e+06
 3.35293911e+01 8.36441214e-05]

2th hidden state
mean =  [6.16368180e-01 1.19324374e+01 4.03423347e-01 7.14512108e+03
 1.44332432e+01 5.26488198e-04]
var =  [2.99003456e-01 2.60774818e+01 1.83409830e-01 6.06682616e+05
 1.32459697e+01 5.55505893e-05]

3th hidden state
mean =  [4.67317073e-01 4.88048780e+01 5.12780488e+01 2.20680244e+04
 3.63041463e+01 4.05096827e-03]
var =  [2.75233849e-03 1.56157050e+02 2.49251996e+03 1.13781173e+06
 5.12520155e+01 4.07395592e-04]



In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=NumRegimes, cols=1)
for i in range(NumRegimes):
    mask = hidden_states == i
    fig.add_trace(go.Scattergl(
        x=pd.to_datetime(select.index.values[mask]),
        y=select[col].values[mask],
        name="regime "+ str(i),
        mode="markers"),row=i+1, col=1)

fig.update_layout(height=600, width=1000, title_text="Hidden Markov (Mixture) Model_Regime Subplots")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [10]:
states = (pd.DataFrame(hidden_states, columns=['states'], index=select.index)
          .join(select, how='outer')
          .assign(mkt_cret=select.sret.cumsum())
          .reset_index(drop=False)
          .rename(columns={'index':'Date'}))
#states.head()

In [11]:
states.tail()

,Date,states,T10Y2Y,T10Y3M,SPY,sret,Ind_Manufac,VIX,Unemployment,recession prob,Recession,mkt_cret
4305,2021-03-26,1,1.53,1.65,395.980011,0.015986,0.000000,18.86,9710,0.00,11,1.471350
4306,2021-03-29,1,1.59,1.70,395.779999,-0.000505,0.000000,20.74,9710,0.00,11,1.470845
4307,2021-03-30,1,1.57,1.71,394.730011,-0.002656,0.000000,19.61,9710,0.00,11,1.468189
4308,2021-03-31,1,1.58,1.71,396.329987,0.004045,0.000000,19.40,9710,0.00,11,1.472234
4309,2021-04-01,2,1.52,1.67,400.609985,0.010741,-0.001017,17.33,9812,0.92,12,1.482975


In [12]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=states["Date"],
    y=states["SPY"].astype(str), 
    marker=dict(
        size=5,
        color=states["states"],
        colorbar=dict(
            title="Colorbar"
        ),
        colorscale="Viridis"
      
    ),
    mode="markers"))
fig.update_layout(title={'text': "Historical SPY Regimes"})
fig.add_vrect(
    x0="2007-10-1", x1="2009-06-30",
    fillcolor="Black", opacity=0.1,
    layer="below", line_width=0,
)
fig.add_vrect(
    x0="2020-02-1", x1="2020-06-30",
    fillcolor="Black", opacity=0.1,
    layer="below", line_width=0,
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()